First, let's define monoids. These are abstract algebras satisfying a few properties. Given a type, X, a monoid over X  is a method for concatenating X's, plus a unit for X, plus a few more properties.

In [2]:
class Monoid():
    def __init__(self, mult, unit):
        self.mult = mult
        self.unit = unit
        # Such that the following equations are true
        # self.mult(x, self.mult(y, z)) == self.mult(self.mult(x, y), z)
        # self.mult(x, self.unit) == self.mult(self.unit, x) == x

    def cmult(self):
        return lambda x: lambda y: self.mult(x, y)

In [47]:
def add(x, y):
    return x + y

assert add(324, add(124, 832)) == add(add(324, 124), 832)
assert add(324, 0) == 324
assert add(0, 324) == 324

additive_monoid = Monoid(add, 0)




def mul(x, y):
    return x * y

assert mul(324, mul(124, 832)) == mul(mul(324, 124), 832)
assert mul(324, 1) == 324
assert mul(1, 324) == 324

multiplicitive_monoid = Monoid(mul, 1)




def str_concat(s1, s2):
    return s1 + s2

assert str_concat("324", str_concat("124", "832")) == str_concat(str_concat("324", "124"), "832")
assert str_concat("324", "") == "324"
assert str_concat("", "324") == "324"

str_concat_monoid = Monoid(str_concat, "")

When placed within a list, there's a systematic way of reducing a list of elements down into a single value.

In [4]:
def reduce(mon, list):
    if list == []:
        return mon.unit
    else:
        return mon.mult(list[0], reduce(mon, list[1:]))

In [45]:
reduce(additive_monoid, [1,2,3,4])

10

It's important to note about this that we're evaluating it (1 + (2 + (3 + 4))). We could evaluate it as (((1 + 2) + 3) + 4). This is on;y gaurenteed to give us the same result because of the monoid laws.

And we can add this as a method to our monoid class.

In [5]:
class Monoid():
    def __init__(self, mult, unit):
        self.mult = mult
        self.unit = unit
        # Such that the following equations are true
        # self.mappend(x, self.mappend(y, z)) == self.mappend(self.mappend(x, y), z)
        # self.mappend(x, self.unit) == self.mappend(self.unit, x) == x

    def cmult(self):
        return lambda x: lambda y: self.mult(x, y)
    
    def reduce(self, list):
        if list == []:
            return self.unit
        else:
            return self.mult(list[0], self.reduce(list[1:]))


In [68]:
additive_monoid.reduce([1,2,3,4])

10

In [69]:
multiplicitive_monoid.reduce([1,2,3,4])

24

Lists themselves form a monoid (in fact, lists are THE free monoid).

In [44]:
def concat(x, y):
    return x + y

assert concat([3,2,4], concat([1,2,4], [8,3,2])) == concat(concat([3,2,4], [1,2,4]), [8,3,2])
assert concat([3,2,4], []) == [3,2,4]
assert concat([], [3,2,4]) == [3,2,4]

free_monoid = Monoid(concat, [])

The reduce operation for this monoid corresponds to list flattening.

In [72]:
free_monoid.reduce([[1,2,3],[4,5],[6,7,8]])

[1, 2, 3, 4, 5, 6, 7, 8]

Some more monoid examples;

In [10]:
def bor(x, y):
    return x or y

assert bor(True, bor(False, True)) == bor(bor(True, False), True)
assert bor(False, False) == False
assert bor(False, True) == True

bool_or_monoid = Monoid(bor, False)

def band(x, y):
    return x and y

assert band(True, band(False, True)) == band(band(True, False), True)
assert band(False, True) == False
assert band(True, True) == True

bool_and_monoid = Monoid(band, True)

reduce for these is the same as .any and .all

In [74]:
bool_or_monoid.reduce([True, False, False, True])

True

In [75]:
bool_and_monoid.reduce([True, False, False, True])

False

Minimum and maximum form a monoid as well where positive and negative infinity act as the units.

In [11]:
from math import inf

assert min(324, min(-124, 832)) == min(min(324, -124), 832)
assert min(-324, inf) == -324
assert min(inf, 324) == 324

minimum_monoid = Monoid(min, inf)

assert max(324, max(-124, 832)) == max(max(324, -124), 832)
assert max(324, -inf) == 324
assert max(-inf, -324) == -324

maximum_monoid = Monoid(max, -inf)

A more interesing example; functions form a monoid under composition.

In [13]:
def comp(f, g):
    return lambda x: f(g(x))

# It's not easy testing that this is a Monoid, so you'll have to trust me.
endofunction_monoid = Monoid(comp, lambda x: x)

In [153]:
endofunction_monoid.reduce([lambda x: x**2, len, lambda x: x+"!"])("Hello")

36

In [78]:
endofunction_monoid.reduce([])("Hello")

'Hello'

On a similar note, procedures form a monoid with sequential execution as composition.

In [12]:
def run(p1, p2):
    p1
    p2
    pass
    
def proc_unit():
    pass

proc_monoid = Monoid(run, proc_unit)

In [131]:
def write_file():
    f=open("hello.txt","w+")
    f.write("Hello!")
    f.close()

def read_file():
    f=open("hello.txt","r")
    print(f.read())
    f.close()

def write_file2():
    f=open("hello.txt","w+")
    f.write("There!")
    f.close()

proc_monoid.reduce([write_file(), read_file(), write_file2(), read_file()])

Hello!
There!


From here we can combine mapping capabilities with reduction.

In [90]:
def map_reduce(mon, f, l):
    return mon.reduce(list(map(f, l)))

We can add this to our class.

In [7]:
class Monoid():
    def __init__(self, mult, unit):
        self.mult = mult
        self.unit = unit
        # Such that the following equations are true
        # self.mappend(x, self.mappend(y, z)) == self.mappend(self.mappend(x, y), z)
        # self.mappend(x, self.unit) == self.mappend(self.unit, x) == x

    def cmult(self):
        return lambda x: lambda y: self.mult(x, y)
    
    def reduce(self, list):
        if list == []:
            return self.unit
        else:
            return self.mult(list[0], self.reduce(list[1:]))

    def map_reduce(self, f, l):
        return self.reduce(list(map(f, l)))

In [102]:
maximum_monoid.map_reduce(len, ["Hi", "Yes", "Hello", "Totally"])

7

There's nothing much interesting to say about it, other than to point out that it maps, then reduces. The fact that the target of the map is a monoid is significant for guaranteeing a sensible answer

A good example of the utility of the map part is in taking means. The mean of two numbers is not an associative operation.

In [14]:
from statistics import mean

mean([mean([2,3]), 4])

3.25

In [15]:
mean([2,mean([3, 4])])

2.75

This is simply because the mean doesn't know how many numbers went into calculating its sub-arguments. If we pair a number with the number of numbers used to calculate the mean, we can get an associative mean. 

In [17]:
def amean(a, b):
    (m, mn) = a
    (n, nn) = b
    return ((m * mn + n * nn) / (mn + nn), mn + nn)

In [18]:
amean(amean((2, 1), (3, 1)), (4, 1))

(3.0, 3)

In [19]:
amean((2, 1), amean((3, 1), (4, 1)))

(3.0, 3)

Using map reduce to map `lambda x: (x, 1)` over a list of numbers, we can use `map_reduce` to get a mean.

In [22]:
mean_monoid = Monoid(amean, (0, 0))

mean_monoid.map_reduce(lambda x: (x, 1), [41,6,2,19,32,43,21])

(23.428571428571427, 7)

On large data, there are more complicated optimizations used that diverge from this design pattern for one reason or another. If we were doing work across several servers, the associativity of the operation gaurentees we can do all our work isolated within those servers before combining the answers afterword without jepordizing the validity of our answer.

There are other abstract algebraic objects that play a large role in data management. Algebird is a library commonly used for data analytics who's design is focused on algebraic structures.

It's worth nothing that lists themselves can be defined like so;

In [ ]:
List A = (m : Monoid X) → (A → X) → X

That is, lists can be defined as monoids paired with a way to map to said monoid. We can define the various list operations like so;

In [27]:
nil = lambda m: lambda f: m.unit

def append(a, l):
    return lambda m: lambda f: m.mult(f(a), l(m)(f))

def concat(l1, l2):
    return lambda m: lambda f: m.mult(l1(m)(f), l2(m)(f))

In [31]:
test_list = concat(
                append(1, append(2, append(3, nil))),
                append(4, append(5, append(6, nil)))
            )

test_list

<function __main__.concat.<locals>.<lambda>(m)>

We can get the length of the list.

In [30]:
test_list(additive_monoid)(lambda x: 1)

6

The mean.

In [42]:
test_list(mean_monoid)(lambda x: (x, 1))

(3.5, 6)

We can define a function which, possibly, returns its first argument. Its behviour allows None to act as the monoid unit.

In [36]:
def maybe_first(a, b):
    if a == None:
        return b
    else:
        return a

Using this, we can get the head of our list.

In [40]:
maybe_first_monoid = Monoid(maybe_first, None)

In [41]:
test_list(maybe_first_monoid)(lambda x: x)

1

We can convert these function-encoded lists into regular lists by simply feeding them the free_monoid, along with a list-monad return function.

In [46]:
test_list(free_monoid)(lambda x: [x])

[1, 2, 3, 4, 5, 6]

So, why would you want to do this? Well, one, it's pretty cool. It's a fact that lambda expressions, entirely on their own, are computationally universal. All data and programs can be encoded by them with literally nothing else. Some programming languages do, in fact, use these representations internally. Cedille and Formality, for example. In the case of formality, lambda expressions directly map onto programming graphs that can be executed in parallel. For example, here's the graph for append.
![alt text](graph.png "Interaction graph for append")

The monoid laws gaurentee that, for instance, + is still associative and empty lists act ampty.

Here's the full, properly formatted proof written in Agda

In [ ]:
postulate funext : {A : Set} {B : A → Set} {f g : (a : A) → B a} → (∀ a → f a ≡ g a) → f ≡ g

record Monoid : Set where
  field
    Carrier : Set
    _∙_ : Carrier → Carrier → Carrier
    𝟙 : Carrier

    ∙-assoc : ∀ {a b c} → a ∙ (b ∙ c) ≡ (a ∙ b) ∙ c
    ∙-𝟙-lcanc : ∀ {a} → a ∙ 𝟙 ≡ a
    ∙-𝟙-rcanc : ∀ {a} → 𝟙 ∙ a ≡ a

open Monoid {{...}}

ForgetMonoid : Monoid → Set
ForgetMonoid = Monoid.Carrier

List : (A : Set) → Set 
List A = ⦃ m : Monoid ⦄ → (A → ForgetMonoid m) → ForgetMonoid m

nil : ∀ {A : Set} → List A
nil ⦃ m ⦄ f = Monoid.𝟙 m

cons : ∀ {A : Set} → A → List A → List A
cons : ∀ {A : Set} → A → List A → ⦃ m : Monoid ⦄ → (A → ForgetMonoid m) → ForgetMonoid m
cons a xs ⦃ m ⦄ f = Monoid._∙_ m (f a) (xs ⦃ m ⦄ f)

_++_ : ∀ {A : Set} → List A → List A → List A
_++_ l1 l2 ⦃ m ⦄ f = Monoid._∙_ m (l1 ⦃ m ⦄ f) (l2 ⦃ m ⦄ f)

++-assoc : {A : Set} {a b c : List A} → (a ++ (b ++ c)) ≡ ((a ++ b) ++ c)
++-assoc {A} {a} {b} {c} = funext (λ f → ∙-assoc)

++-nil-lcanc : {A : Set} {a : List A} → (a ++ nil) ≡ a
++-nil-lcanc {A} {a} = funext (λ m → funext (λ f → Monoid.∙-𝟙-lcanc m))

++-nil-rcanc : {A : Set} {a : List A} → (nil ++ a) ≡ a
++-nil-rcanc {A} {a} = funext (λ m → funext (λ f → Monoid.∙-𝟙-rcanc m))
